In [0]:
%pip install xgboost pandas mlflow

In [0]:
import mlflow.xgboost
import pandas as pd
import numpy as np
from pyspark.sql.functions import col
import json

In [0]:
# Se você registrou o modelo como 'seu_catalog.seu_schema.churn_risk_model'
model_name = "workspace.churn_zero.churn_risk_model_xgboost" 
loaded_model = mlflow.xgboost.load_model(f"models:/{model_name}@n2") 

print("Modelo carregado do Model Registry/Unity Catalog.")

In [0]:
with open("/Workspace/Users/joaovab@al.insper.edu.br/Hackathon_Terra_Signal_grupo_1/feature_names.json", "r") as f:
    feature_names = json.load(f)

print("Features carregadas com sucesso!")
print(feature_names)

In [0]:
df_inferencia_spark = spark.table("workspace.churn_zero.inference_gold")
df_inferencia_pandas = df_inferencia_spark.toPandas()

df_inferencia_pandas['feedback_topic_service_quality'] = 0

customer_ids = df_inferencia_pandas['customerID']

# Remove 'customerID' and 'MonthlyCharges' from feature_names before selection
selected_features = [
    col for col in feature_names
    if col not in ['customerID', 'MonthlyCharges']
]
X_inferencia = df_inferencia_pandas[selected_features]

In [0]:
for c in df_inferencia_pandas.columns:
    if c not in feature_names:
        print(c)

In [0]:
churn_probabilities = loaded_model.predict_proba(X_inferencia)[:, 1]

# Crie o DataFrame Pandas com as duas colunas-chave
results_df_pandas = pd.DataFrame({
    'customerID': df_inferencia_pandas['customerID'], # Use o ID do cliente como chave
    'churn_probability': churn_probabilities
})

# --- 2. Converter o DF Pandas para Spark ---
# Esta conversão é necessária para realizar o JOIN com a tabela Spark original.
results_df_spark = spark.createDataFrame(results_df_pandas)

print("DataFrame de resultados em formato Spark criado.")
results_df_spark.show(5)

In [0]:
# 1. Carregar a Tabela Original de Inferência (Assumindo que contém todas as colunas originais)
tabela_original = spark.table("workspace.churn_zero.inference_gold") 

# 2. Realizar o JOIN (Inner Join)
# O join é baseado no customerID, adicionando a coluna 'churn_probability' à tabela original
df_com_probabilidade = tabela_original.join(
    results_df_spark,
    on="customerID",
    how="inner" # Garante que apenas clientes presentes em ambos os lados sejam mantidos
)

# 3. Visualizar o resultado final
print("\nPrévia da Tabela Original com a Coluna de Probabilidade Adicionada:")
df_com_probabilidade.select("customerID", "Partner", "Churn_Probability").show(5)

# 4. Salvar a Tabela Final
# Substitua pelos nomes de destino
CATALOGO_FINAL = "workspace"
SCHEMA_FINAL = "churn_zero"
TABELA_INFERENCIA_FINAL = "inference_churn"

df_com_probabilidade.write.mode("overwrite").saveAsTable(f"{CATALOGO_FINAL}.{SCHEMA_FINAL}.{TABELA_INFERENCIA_FINAL}")
print(f"\nTabela final salva em: {TABELA_INFERENCIA_FINAL}")